In [162]:
import pydicom as dicom
import os
import cv2
import PIL # optional
import pandas as pd
import shutil
folder_path = "/Users/jacksimonson/Springboard_Work/data/"
label_path = '/Users/jacksimonson/Springboard_Work/labels_raw_csv/'
parent_path = ['calc-train', 'mass-train']
files = ['Calc-Training.csv', 'Mass-Training.csv']
data = {file: pd.read_csv(os.path.join(label_path, file)) for file in files}
for file, df in data.items():
    
    mask_paths = df['ROI mask file path']
    roi_paths = df['cropped image file path']
    full_image_paths = df['image file path']
    pathology = df['pathology']
    
    for index, row in df.iterrows():        
        mask = row['ROI mask file path']
        roi = row['cropped image file path']
        full_image = row['image file path']
        pathology = row['pathology']
        if pathology == 'BENIGN_WITHOUT_CALLBACK':
            continue
        for name, img_type in {'mask':mask, 'roi':roi, 'image':full_image}.items():
            suffix = 0
            full_dir = '/'.join(img_type.split('/')[:-1])
            parent = file[:4].lower() + '-test' if 'test' in mask.lower() else file[:4].lower() + '-train'
            full_dir = os.path.join(folder_path, os.path.join(parent, full_dir))

            # delete duplicates
            try:
                suffix = int(img_type.split('/')[0][-1])
                if suffix > 1:
                    shutil.rmtree(os.path.join(folder_path, mask.split('/')[0]))
                    continue
            except:
                pass

            for f in os.listdir(full_dir):
                img = os.path.join(full_dir, f)
                
                if suffix == 1:
                    if name == "mask" and f.endswith('-2.dcm'):
                        child_path = file.lower()[:-4] + '-mask'
                        child_path = child_path[5:]
                        new_img = os.path.join(os.path.join(folder_path, child_path) + '/{}_{}'.format(full_dir.split('/')[6][:-2], pathology)) + '.png'
                    elif name == "roi" and f.endswith('-1.dcm'):
                        child_path = file.lower()[:-4] + '-roi'
                        child_path = child_path[5:]
                        new_img = os.path.join(os.path.join(folder_path, child_path) + '/{}_{}'.format(full_dir.split('/')[6][:-2], pathology)) + '.png'
                    else:
                        continue
                else:
                    child_path = file.lower()[:-4] + '-image'
                    child_path = child_path[5:]
                    new_img = os.path.join(os.path.join(folder_path, child_path) + '/{}_{}'.format(full_dir.split('/')[6], pathology)) + '.png'

                ds = dicom.dcmread(img, force = True)
                pixel_array_numpy = ds.pixel_array
                new_img = new_img.replace('.dcm', '.png')
                cv2.imwrite(new_img, pixel_array_numpy)
